# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
   file_path_list = glob.glob(os.path.join(root.replace(".ipynb_checkpoints",""),'*')) 
    #(root,'*'))
#print(file_path_list)


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:

# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
  

    
# for every filepath in the file path list 
for f in file_path_list:
        
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
   
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    #count=1
    #print(full_data_rows_list)
    for row in full_data_rows_list:
            
        if (row[0] == ''):
            continue
        else:
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
            #print(count)
            #count=count+1
                
                
                
            
    

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


 

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# a session to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

# Part II. Queries
### There are 3 tables created for 3 queries:
1. query the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. query the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. query every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [8]:
## Query 1:  query the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## create table song_info_by_session, set session_id, item_in_session as primary key

query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query = query + "(session_id int, item_in_session int,\
                    artist text, length float, \
                     song text, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
## create file event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert data into the song_info_by_sessio table
        query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist, length, song)"
                                             
        query = query + " VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

In [10]:
## query to verify the data was entered into the table
query = "select artist, song, length from song_info_by_session WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
## Query 2: query the name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## create table song_info_by_userID_and_sessionID, set user_id and session_id as partition key and item_in_session as clustering column
query = "CREATE TABLE IF NOT EXISTS song_info_by_user_and_session"
query = query + "(user_id int,  session_id int, item_in_session int, artist text, first_name text, last_name text, \
                   song text, PRIMARY KEY ((user_id, session_id), item_in_session))"
                     
try:
    session.execute(query)
except Exception as e:
    print(e)

    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT datab into table song_info_by_user_and_session
        query = "INSERT INTO song_info_by_user_and_session (user_id, session_id, item_in_session, artist, first_name, last_name, song)"
                                             
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9], ))
                    

In [12]:
## query to verify the data was entered into the table
query = "select artist, song, first_name, last_name from song_info_by_user_and_session WHERE user_id=10 AND session_id=182 ORDER BY item_in_session ASC"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
## query the user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## create table song_info_by_songName, set song and user_id as primary key
query = "CREATE TABLE IF NOT EXISTS song_info_by_songName"
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"
                   
                     
try:
    
    session.execute(query)
except Exception as e:
    print(e)

    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## insert the data into table song_info_by_songName
        query = "INSERT INTO song_info_by_songName (song, user_id, first_name, last_name)"
                                             
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [14]:
## query to verify the data was entered into the table
query = "select first_name, last_name from song_info_by_songName WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [15]:
## Drop the table before closing out the sessions

In [16]:
query1 = "drop table IF EXISTS song_info_by_session"
query2 = "drop table IF EXISTS song_info_by_user_and_session"
query3 = "drop table IF EXISTS song_info_by_songName"
try:
    rows = session.execute(query1)
    rows = session.execute(query2)
    rows = session.execute(query3)
except Exception as e:
    print(e)
    


### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()